In [63]:
import os
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm
import re
from collections import Counter


In [39]:
def short_name(team):
        short_name={'Chennai Super Kings':'CSK',
    'Mumbai Indians':'MI',
    'Gujarat Titans':'GT',
    'Kolkata Knight Riders':'KKR',
    'Punjab Kings':'PBKS',
    'Sunrisers Hyderabad':'SRH',
    'Rajasthan Royals':'RR',
    'Lucknow Super Giants':'LSG',
    'Delhi Capitals':'DC',
    'Royal Challengers Bangalore':'RCB'
    }
        return short_name[team]

In [40]:
def get_moth(url):
    cricbuzz_highlights_url=url
    cricbuzz_highlights_url=cricbuzz_highlights_url.replace('cricket-full-commentary','cricket-scores')
    cricbuzz_highlights_url
    page=requests.get(cricbuzz_highlights_url)
    soup=BeautifulSoup(page.content)
    try: 
        win=soup.find('div',{'class':'cb-col cb-col-100 cb-min-stts cb-text-complete'}).text
        #cb-col cb-col-100 cb-mini-col cb-min-comp ng-scope
        #cb-link-undrln ng-binding
        a=soup.find('span',{'class':'cb-text-gray cb-font-12'})
        try:
            man_of_the_match=a.next_sibling.next_sibling.next_sibling.text
        except:
            man_of_the_match=None
    except:
        win = None
        man_of_the_match=None
    file_name=url.split('/')[-1]
    df=pd.read_csv(f'files/2021/{file_name}.csv')
    df['win']=win
    df['man_of_the_match']=man_of_the_match
    df.to_csv(f'files/2021/{file_name}.csv',index=False)

In [41]:
def find_toss(url):
    file_name=url.split('/')[-1]
    df=pd.read_csv(f'files/2023/{file_name}.csv')
    try :
        toss=df['toss'][0]
        toss_win=toss[:toss.find('have')]
        toss_choose=toss.split(' ')[-1]

        df['Toss_winner']=short_name(toss_win.strip())
        df['toss_chosen']=toss_choose
        df.to_csv(f'files/2023/{file_name}.csv',index=False)
    except:
        print('error')
        pass
#find_toss('https://www.cricbuzz.com/cricket-full-commentary/35612/mi-vs-rcb-1st-match-indian-premier-league-2021')

In [42]:

def find_innings(url):
    file_name=url.split('/')[-1]
    #print(file_name)
    df=pd.read_csv(f'files/2023/{file_name}.csv')
       
    try :
        toss_winner=df['Toss_winner'][0]
        #print(toss_winner)
        toss_choose=df['toss_chosen'][0]
        #print(toss_choose)
        if toss_choose=='field':
            #print('fielding')
           
            #print(df[df['innings']== f'{toss_winner} Inns'.strip()]['innings'])
            df['innings'].mask(df['innings']!=f'{toss_winner} Inns'.strip(),'1st Inns',inplace=True)
            df['innings'].mask(df['innings']==f'{toss_winner} Inns'.strip(),'2nd Inns',inplace=True)
            
            #print(df)
        else:
            df['innings'].mask(df['innings']!=f'{toss_winner} Inns'.strip(),'2nd Inns',inplace=True)
            df['innings'].mask(df['innings']==f'{toss_winner} Inns'.strip(),'1st Inns',inplace=True)
            
            
           
        #display(df)
        df.to_csv(f'files/2023/{file_name}.csv',index=False)
    except:
        print('error')

#find_innings('https://www.cricbuzz.com/cricket-full-commentary/66442/csk-vs-kkr-61st-match-indian-premier-league-2023')

In [43]:
def find_bowling_team(url):
    file_name=url.split('/')[-1]
    #print(file_name)
    df=pd.read_csv(f'files/2023/{file_name}.csv')
    try :
        toss_winner=df['Toss_winner'][0]
        toss_choose=df['toss_chosen'][0]
        a_team=short_name(df['team_a'][0])
        b_team=short_name(df['team_b'][0])
        teams=[a_team,b_team]
        for i in teams:
            if i==toss_winner:
                teams.remove(i)
        opt=teams[0]
        if toss_choose=='bat':
            df.loc[df['innings']=='1st Inns','batting_team']=toss_winner
            df.loc[df['innings']=='1st Inns','bowling_team']=opt
            df.loc[df['innings']=='2nd Inns','batting_team']=opt
            df.loc[df['innings']=='2nd Inns','bowling_team']=toss_winner
        if toss_choose=='field':
            df.loc[df['innings']=='1st Inns','batting_team']=opt
            df.loc[df['innings']=='1st Inns','bowling_team']=toss_winner
            df.loc[df['innings']=='2nd Inns','batting_team']=toss_winner
            df.loc[df['innings']=='2nd Inns','bowling_team']=opt
            
        #display(df)
        df.to_csv(f'files/2023/{file_name}.csv',index=False)
    except Exception as e:
        print(e)
       
#find_bowling_team('https://www.cricbuzz.com/cricket-full-commentary/66519/kkr-vs-gt-39th-match-indian-premier-league-2023')

In [44]:
def find_batsman_bowler(url):
    file_name=url.split('/')[-1]
    #print(file_name)
    df=pd.read_csv(f'files/2023/{file_name}.csv')
    df['Batsman']=''
    df['Bowler']=''
    #display(df)
    for i,row in enumerate(df['Commentary Text'].values):
        text=row.split(',')[0]
        text=text.split(' to ')
        batsman=text[1]
        bowler=text[0]
        df.loc[i,['Batsman']]=batsman
        #df['Batsman'][i]=batsman
        df.loc[i,['Bowler']]=bowler

        #df['Bowler'][i]=bowler
    #display(df)
    df.to_csv(f'files/2023/{file_name}.csv',index=False)
        

#find_batsman_bowler('https://www.cricbuzz.com/cricket-full-commentary/35612/mi-vs-rcb-1st-match-indian-premier-league-2021')

In [45]:
texts=[]
#find the runs by ball 
def find_runs(url):
    file_name=url.split('/')[-1]
    #print(file_name)
    df=pd.read_csv(f'files/2021/{file_name}.csv')
    df['runs']=''
    df['extra']=''
    for i,comment in enumerate(df['Commentary Text'].values):
        text=comment.split(',')[1].strip().lower()
        #print('text=',text)
        texts.append(text)
        if text.startswith('1 run'.casefold()):
            df.loc[i,['runs']]= 1
            df.loc[i,['extra']]= 0
        elif text.startswith('2 runs'.casefold()):
            text= 2
            df.loc[i,['runs']]= 2
            df.loc[i,['extra']]= 0
        elif text.startswith('no run'.casefold()):
            df.loc[i,['runs']]= 0
            df.loc[i,['extra']]= 0
        elif text.startswith('3 runs'.casefold()):
            df.loc[i,['runs']]= 3
            df.loc[i,['extra']]= 0
        elif text.startswith('FOUR'.casefold()):
            df.loc[i,['runs']]= 4
            df.loc[i,['extra']]= 0
        elif text.startswith('SIX'.casefold()):
            df.loc[i,['runs']]= 6
            df.loc[i,['extra']]= 0
        elif text.startswith(('byes'.casefold(),'leg'.casefold())):
            if str(comment.split(',')[2]).lstrip().lower()=='four':
                df.loc[i,['extra']]= 4
            elif str(comment.split(',')[2]).lstrip().lower()=='six':
                df.loc[i,['extra']]= 6
            elif str(comment.split(',')[2]).lstrip().lower()=='1 run':
                df.loc[i,['extra']]= 1
            elif str(comment.split(',')[2]).lstrip().lower()=='2 runs':
                df.loc[i,['extra']]= 2
            elif str(comment.split(',')[2]).lstrip().lower()=='3 runs':
                df.loc[i,['extra']]= 3
            else:
                df.loc[i,['extra']]=0
            df.loc[i,['runs']]= 'byes'
        elif text.startswith('no ball'.casefold()):
            if str(comment.split(',')[2]).lstrip().lower()=='four':
                df.loc[i,['extra']]= '1+4'
            elif str(comment.split(',')[2]).lstrip().lower()=='six':
                df.loc[i,['extra']]='1+6'
            elif str(comment.split(',')[2]).lstrip().lower()=='1 run':
                df.loc[i,['extra']]= 1
            elif str(comment.split(',')[2]).lstrip().lower()=='2 runs':
                df.loc[i,['extra']]= 2
            elif str(comment.split(',')[2]).lstrip().lower()=='3 runs':
                df.loc[i,['extra']]= 3
            else:
                df.loc[i,['extra']]=1
            df.loc[i,['runs']]= 'no ball'
        elif text.startswith('out'.casefold()):
            df.loc[i,['runs']]= 'out'
            df.loc[i,['extra']]= 'out'
        elif text.startswith('wide'.casefold()):
            df.loc[i,['runs']]= 'wide'
            df.loc[i,['extra']]= 1
        elif text.startswith('2 wides'.casefold()):
            df.loc[i,['runs']]= 'wide'
            df.loc[i,['extra']]= 2
        elif text.startswith('3 wides'.casefold()):
            df.loc[i,['runs']]= 'wide'
            df.loc[i,['extra']]= 3
        elif text.startswith('4 wides'.casefold()):
            df.loc[i,['runs']]= 'wide'
            df.loc[i,['extra']]= 4
        elif text.startswith('5 wides'.casefold()):
            df.loc[i,['runs']]= 'wide'
            df.loc[i,['extra']]= 5
        
        #print(text)
    #display(df.sample(5))
    df.to_csv(f'files/2021/{file_name}.csv',index=False)

find_runs('https://www.cricbuzz.com/cricket-full-commentary/35612/mi-vs-rcb-1st-match-indian-premier-league-2021')
#for i in set(texts):
    #print(i)

In [96]:
# find the length of the each ball
def find_length(url):
    file_name=url.split('/')[-1]
    #print(file_name)
    length=[]
    none=[]
    df=pd.read_csv(f'files/2021/{file_name}.csv')
    #display(df)
    for i,line in enumerate(df["Commentary Text"]):
        match=re.search('short|full-toss|back of|good|tosse|full|on a length|yorker|of a length|length delivery|length ball|length on',line,re.IGNORECASE)

        length_balls=['on a length','of a length','length delivery','length ball','length on']
        
        if match:
            
            if match.group(0).casefold()=='short':
                df.loc[i,['ball_length']]= 'Short'
            elif match.group(0).casefold()=='full':
                df.loc[i,['ball_length']]= 'Full'
            elif match.group(0).casefold()=='back of':
                df.loc[i,['ball_length']]= 'Back of Length'
            elif match.group(0).casefold()=='good':
                df.loc[i,['ball_length']]= 'Good'
            elif match.group(0).casefold()=='tosse' or match.group(0).casefold()=='full-toss':
                df.loc[i,['ball_length']]= 'Full toss'
            elif match.group(0).casefold()=='yorker':
                df.loc[i,['ball_length']]= 'yorker'
            elif  match.group(0).casefold() in length_balls:
                df.loc[i,['ball_length']]= 'Good'
            else:
                print(match.group(0))
                break
                df.loc[i,['ball_length']]= 'error'

        else:
            df.loc[i,['ball_length']]= 'undefined'
    #display(df)
    df.to_csv(f'files/2021/{file_name}.csv',index=False)    
#find_length('https://www.cricbuzz.com/cricket-full-commentary/35612/mi-vs-rcb-1st-match-indian-premier-league-2021')

In [118]:
#used to check the commentry
def check_commentary(url):
    file_name=url.split('/')[-1]
    #print(file_name)
    ball_line=[]
    none=[]
    df=pd.read_csv(f'files/2021/{file_name}.csv')
    print(df.info())
    for i,line in enumerate(df['Commentary Text']):
        match=re.search('down-leg|outside off|on the stumps|on the pads|on the middle|on the leg|into off|off',line,re.IGNORECASE)
        if match:
            ball_line.append(line)
    print(len(ball_line))
#check_commentary('https://www.cricbuzz.com/cricket-full-commentary/35612/mi-vs-rcb-1st-match-indian-premier-league-2021')

In [119]:
def check_commentary(url):
    file_name=url.split('/')[-1]
    #print(file_name)
    t=[]
    df=pd.read_csv(f'files/2021/{file_name}.csv')
    for i,line in enumerate(df['Commentary Text']):
        if 'outside off stump' in line.lower():
            pass
        elif 'off stump' in line.lower():
            pass
        elif 'outside off' in line.lower():
            pass
        elif 'off the pads' in line.lower():
            pass
        elif 'of off' in line.lower():
            pass
        elif 'off' in line.lower():
            print(line)
            t.append(line)

    print(len(t))
#check_commentary('https://www.cricbuzz.com/cricket-full-commentary/35612/mi-vs-rcb-1st-match-indian-premier-league-2021')

In [142]:
def find_wicket(url):
    file_name=url.split('/')[-1]
    #print(file_name)
    df=pd.read_csv(f'files/2021/{file_name}.csv')
    #print(df.info())
    #display(df)
    for i,line in enumerate(df['runs']):
        if line=='out':
            text=df.loc[i,'Commentary Text']
            text=text.split('.')[-1]
            print(text)
            match=re.search(r'\b+c\s(\w+)',text)
            if match:
                print(match)
            #print(text)
            #display(df[df['runs']=='out'])

find_wicket('https://www.cricbuzz.com/cricket-full-commentary/35612/mi-vs-rcb-1st-match-indian-premier-league-2021')

 Rahul Chahar run out (Kohli/de Villiers) 0(0) 


error: nothing to repeat at position 2

In [95]:
links=[]
with open('match_links/2023_season_match_links.txt','r') as file:
    for line in file:
        links.append(line.strip())
print(links)
links=links[74:]
for i in tqdm(links,colour='green'):
    #print(i)
    find_length(i)

['https://www.cricbuzz.com/cricket-full-commentary/66169/gt-vs-csk-1st-match-indian-premier-league-2023', 'https://www.cricbuzz.com/cricket-full-commentary/66173/pbks-vs-kkr-2nd-match-indian-premier-league-2023', 'https://www.cricbuzz.com/cricket-full-commentary/66176/lsg-vs-dc-3rd-match-indian-premier-league-2023', 'https://www.cricbuzz.com/cricket-full-commentary/66183/srh-vs-rr-4th-match-indian-premier-league-2023', 'https://www.cricbuzz.com/cricket-full-commentary/66190/rcb-vs-mi-5th-match-indian-premier-league-2023', 'https://www.cricbuzz.com/cricket-full-commentary/66197/csk-vs-lsg-6th-match-indian-premier-league-2023', 'https://www.cricbuzz.com/cricket-full-commentary/66204/dc-vs-gt-7th-match-indian-premier-league-2023', 'https://www.cricbuzz.com/cricket-full-commentary/66208/rr-vs-pbks-8th-match-indian-premier-league-2023', 'https://www.cricbuzz.com/cricket-full-commentary/66211/kkr-vs-rcb-9th-match-indian-premier-league-2023', 'https://www.cricbuzz.com/cricket-full-commentary/

100%|██████████| 1/1 [00:00<00:00, 11.12it/s]
